In [5]:
import tensorflow as tf
import numpy as np

# ELU Activation Function

In [44]:
from tensorflow.examples.tutorials.mnist import input_data
mnist =input_data.read_data_sets('./my_datasets_mnist/')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./my_datasets_mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./my_datasets_mnist/train-labels-idx1-ubyte.gz
Extracting ./my_datasets_mnist/t10k-images-idx3-ubyte.gz
Extracting ./my_datasets_mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [45]:
n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_hidden3=30
n_outputs=10

In [46]:
X=tf.placeholder(tf.float32,shape=(None,n_inputs),name='X')
y=tf.placeholder(tf.int64,shape=(None),name='y')

In [47]:
#from tensorflow.contrib.layers import tf.layers.dense
with tf.name_scope('dnn'):
    hidden1=tf.layers.dense(X,n_hidden1,activation=tf.nn.elu)
    hidden2=tf.layers.dense(hidden1,n_hidden2,activation=tf.nn.elu)
    hidden3=tf.layers.dense(hidden2,n_hidden3,activation=tf.nn.elu)
    logits=tf.layers.dense(hidden3,n_outputs)

In [48]:
with tf.name_scope('loss'):
    X_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(X_entropy,name='loss')

In [54]:
learning_rate=0.01
with tf.name_scope('train'):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)

In [55]:
with tf.name_scope('eval'):
    correct=tf.equal(tf.argmax(logits,axis=1),y)
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [56]:
init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [57]:
n_epochs=30
batch_size=100

In [58]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch,y_batch=mnist.train.next_batch(batch_size)
            training_op.run(feed_dict={X:X_batch,y:y_batch})
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,
                                         y:mnist.test.labels})
        print(epoch,'Train acc:',acc_train,'Test acc:',acc_test)
    save_path = saver.save(sess, "./saved_models/dnn_with_elu_on_mnist.ckpt")

0 Train acc: 0.86 Test acc: 0.8754
1 Train acc: 0.84 Test acc: 0.9037
2 Train acc: 0.88 Test acc: 0.9155
3 Train acc: 0.93 Test acc: 0.9205
4 Train acc: 0.92 Test acc: 0.9263
5 Train acc: 0.98 Test acc: 0.9291
6 Train acc: 0.96 Test acc: 0.9307
7 Train acc: 0.99 Test acc: 0.9356
8 Train acc: 0.95 Test acc: 0.9372
9 Train acc: 0.94 Test acc: 0.9408
10 Train acc: 0.95 Test acc: 0.9428
11 Train acc: 0.94 Test acc: 0.9437
12 Train acc: 0.9 Test acc: 0.9449
13 Train acc: 0.97 Test acc: 0.9473
14 Train acc: 0.98 Test acc: 0.9476
15 Train acc: 0.95 Test acc: 0.95
16 Train acc: 0.92 Test acc: 0.9512
17 Train acc: 0.95 Test acc: 0.9524
18 Train acc: 0.95 Test acc: 0.9544
19 Train acc: 0.94 Test acc: 0.9546
20 Train acc: 0.98 Test acc: 0.9554
21 Train acc: 0.99 Test acc: 0.9578
22 Train acc: 0.94 Test acc: 0.9574
23 Train acc: 0.97 Test acc: 0.9593
24 Train acc: 0.96 Test acc: 0.9594
25 Train acc: 0.93 Test acc: 0.9603
26 Train acc: 0.98 Test acc: 0.9604
27 Train acc: 0.97 Test acc: 0.9622
28 Tr

# Batch Normalization

In [1]:
import tensorflow as tf
import numpy as np

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)


## partial

For programs written in a functional style, you’ll sometimes want to construct variants of existing functions that have some of the parameters filled in. Consider a Python function f(a, b, c); you could create a new function g(b, c) that was equivalent to f(1, b, c)

In [10]:
import functools
def f(a,b,c):
    print(a+b+c)
    
g=functools.partial(f,a=1)
g(b=2,c=2)

5


In [5]:
import functools

def log (message, subsystem):
    "Write the contents of 'message' to the specified subsystem."
    print('%s: %s' % (subsystem, message))
    

server_log = functools.partial(log, subsystem='server')
server_log('Unable to open socket')

server: Unable to open socket


## MNIST

In [3]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [30]:
def generate_batch(X, labels, batch_size):
    rnd_indices=np.random.randint(0,len(X),batch_size)
    return X[rnd_indices], labels[rnd_indices]

In [42]:
from functools import partial

#tf.reset_default_graph()
reset_graph()

mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()
X_train, X_test = X_train.reshape(-1,28*28) / 255.0, X_test.reshape(-1,28*28) / 255.0

batch_norm_momentum=0.9
n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_hidden3=30
n_outputs=10

X=tf.placeholder(tf.float32,(None,n_inputs),'X')
y=tf.placeholder(tf.int32,(None),'y')
training=tf.placeholder_with_default(False,(),'training')

with tf.name_scope('dnn'):
    he_init=tf.variance_scaling_initializer()

    my_batch_norm_layer=partial(
        tf.layers.batch_normalization,
        training=training,
        momentum=batch_norm_momentum)

    my_dense_layer=partial(
        tf.layers.dense,
        kernel_initializer=he_init)
    
    hidden1=my_dense_layer(X,n_hidden1,name='hidden1')#no activation function
    #activation: Set it to None to maintain a linear activation.
    bn1=tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2=my_dense_layer(bn1,n_hidden2,name='hidden2')
    bn2=tf.nn.elu(my_batch_norm_layer(hidden2))
#    hidden3=my_dense_layer(bn2,n_hidden3,name='hidden3')
#    bn3=tf.nn.elu(my_batch_norm_layer(hidden3))
    logits_before=my_dense_layer(bn2,n_outputs,name='outputs')
    logits=my_batch_norm_layer(logits_before)
    
with tf.name_scope('loss'):
    X_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss=tf.reduce_mean(X_entropy,name='loss')

learning_rate=0.01
with tf.name_scope('train'):
    optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op=optimizer.minimize(loss)

with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
   # correct=tf.equal(y,tf.argmax(logits,axis=1))
   # accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

init=tf.global_variables_initializer()
saver=tf.train.Saver()

epochs=20
batch_size=50
with tf.Session() as sess:
    init.run()
    for epoch in range(epochs):
#        for iteration in range(20):
#            X_batch,y_batch=generate_batch(X_train,y_train,batch_size)

        for X_batch,y_batch in shuffle_batch(X_train,y_train,batch_size):
            training_op.run(feed_dict={X:X_batch,y:y_batch})
        print(epoch,'Test accuracy:',accuracy.eval(feed_dict={X:X_test,y:y_test}))
    save_path=saver.save(sess,'./saved_models/batch_norm_on_mnist.ckpt')

0 Test accuracy: 0.9574
1 Test accuracy: 0.9648
2 Test accuracy: 0.9684
3 Test accuracy: 0.9736
4 Test accuracy: 0.9744
5 Test accuracy: 0.9744
6 Test accuracy: 0.9734
7 Test accuracy: 0.9786
8 Test accuracy: 0.9779
9 Test accuracy: 0.9746
10 Test accuracy: 0.9778
11 Test accuracy: 0.9807
12 Test accuracy: 0.9797
13 Test accuracy: 0.9801
14 Test accuracy: 0.9769
15 Test accuracy: 0.9806
16 Test accuracy: 0.9797
17 Test accuracy: 0.9794
18 Test accuracy: 0.9787
19 Test accuracy: 0.9747
